### Random Forest Algorithm, as I understand it

Don't expect it to be perfect, or even good

In [1]:
import pandas as pd
import numpy as np
import math
import csv
import copy

In [2]:
#get gini score for grouping
def get_gini(groups,classes):
    #groups - each element is [feature 1, feature 2,..., class]
    #classes- list of unique classes in data
    gini = 0.0
    total_size = float(sum([len(group) for group in groups]))
    for group in groups:
        group_size = float(len(group))
        if group_size==0.0:
            continue
        score = 0.0
        class_freq = [[member[-1] for member in group].count(cl) for cl in classes]
        class_probs = [float(freq)/group_size for freq in class_freq]
        group_gini = sum([(prob*(1.0-prob)) for prob in class_probs])
        gini+=group_gini*(group_size/total_size)
    return gini

#binary split data into classes based on feature at ind with best gini
def optimal_split_feature(data,classes,ind):
    assert ind<len(data[0])-1#ensure ind is valid feature index
    
    best_gini = 2.0
    best_threshold = data[0][ind]
    best_groups = []
    for member in data:
        left_group = [row for row in data if row[ind]<member[ind]]
        right_group = [row for row in data if row[ind]>=member[ind]]
        gini = get_gini([left_group,right_group],classes)
        if gini<best_gini:
            best_gini = gini
            best_threshold = member[ind]
            best_groups = [left_group,right_group]
    return best_threshold,best_gini,best_groups

#find optimal feature+split
def optimal_split_data(data,classes,features):
    best_gini = 2.0
    best_threshold = 0.0
    best_feature = 0
    best_groups = []
    
    for feature_ind in features:
        threshold, gini, groups = optimal_split_feature(data,classes,feature_ind)
        if gini<best_gini:
            best_gini = gini
            best_threshold = threshold
            best_feature = feature_ind
            best_groups = groups
    
    return best_feature, best_threshold, best_groups

#choose random subset of data members
def get_random_tree_data(data,fraction = 0.7):
    #fraction - expected fraction of data to use
    tree_data = [row for row in data if np.random.uniform()<fraction]
    return tree_data

class DecisionTree:
    
    def __init__(self, features,classes,max_depth):
        #features - list of feature indices to use when building tree
        #classes - list of available classes
        self.features = features
        self.classes = classes
        self.max_depth = max_depth
        
        self.decision_tree = []
    
    #build tree using data
    def build_tree(self,data):
        
        def recursive_build(depth,rem_data):
            if len(rem_data)==0:
                return self.classes[0]#if there are no data left, return arbitrary class. This is probably bad
            if rem_data==1 or depth>=self.max_depth:
                class_mode = self.classes[0]
                mode_freq = 0
                for class_val in self.classes:
                    class_freq = [member[-1] for member in rem_data].count(class_val)
                    if class_freq>mode_freq:
                        class_mode = class_val
                        mode_freq = class_freq
                return class_mode
            
            best_feature,best_threshold,best_groups = optimal_split_data(data,self.classes,self.features)
            l_node = recursive_build(depth+1,best_groups[0])
            r_node = recursive_build(depth+1,best_groups[1])
            return [best_feature,best_threshold,l_node,r_node]
        
        self.decision_tree = recursive_build(0,data)
    
    #get item prediction from tree
    def query_tree(self,item):
        rem_tree = copy.deepcopy(self.decision_tree)
        while type(rem_tree) is list:
            if item[rem_tree[0]]<rem_tree[1]:
                rem_tree = rem_tree[2]
            else:
                rem_tree = rem_tree[3]
        return rem_tree


    
class RandomForest:
    
    def __init__(self,data,n_trees,max_depth=7):
        self.classes = list(set([member[-1] for member in data]))
        
        self.trees = []
        features_per_tree = int(math.sqrt(len(data[0])-1)+0.5)
        for t in range(n_trees):
            feature_indices = np.random.randint(0,len(data[0])-1,size=(features_per_tree,))
            self.trees.append(DecisionTree(feature_indices,self.classes,max_depth))
            self.trees[-1].build_tree(get_random_tree_data(data))
    
    #get item prediction from forest
    def query_forest(self,item):
        tree_guesses = [tree.query_tree(item) for tree in self.trees]
        class_mode = self.classes[0]
        mode_freq = 0
        for class_val in self.classes:
            class_freq = tree_guesses.count(class_val)
            if class_freq>mode_freq:
                class_mode = class_val
                mode_freq = class_freq
        return class_mode
    
    #evaluate forest performance on set of data
    def eval_forest(self,data):
        predictions = [self.query_forest(member) for member in data]
        correct = 0
        for i in range(len(data)):
            if predictions[i]==data[i][-1]:
                correct+=1
        print(f'{correct} out of {len(data)} ({100.0*float(correct)/float(len(data))}%) correct, with {len(self.trees)} trees')
    
    
    
    
    
    
    

In [3]:
data = []
with open('sonar.all-data') as f:
    reader = csv.reader(f)
    for row in reader:
        data_row = [float(entry.strip()) for entry in row[:-1]]
        data_row.append(row[-1].strip())#class
        data.append(data_row)

In [4]:
forests = [RandomForest(data,n_trees) for n_trees in [1,3,5,10]]

In [5]:
for forest in forests:
    forest.eval_forest(data)

144 out of 208 (69.23076923076923%) correct, with 1 trees
143 out of 208 (68.75%) correct, with 3 trees
159 out of 208 (76.4423076923077%) correct, with 5 trees
163 out of 208 (78.36538461538461%) correct, with 10 trees
